In [4]:
import re
import time
import json
import operator
import pandas as pd
import numpy as np
from ast import literal_eval
from scipy import spatial

In [5]:
movies = pd.read_csv('movies_metadata.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
movies.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.85949,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.38752,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [7]:
credits = pd.read_csv('credits.csv')

In [8]:
credits.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [9]:
# movies['genres'] = movies['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [10]:
movies['genres'] = movies['genres'].fillna('[]').apply(literal_eval)

In [11]:
for index,i in zip(movies.index,movies['genres']):
  temp_list1 = []
  for j in range(len(i)):
    temp_list1.append((i[j]['name'])) # the key 'name' contains the name of the genre
  movies.loc[index,'genres'] = str(temp_list1)

In [12]:
links_small = pd.read_csv('links_small.csv')
links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')

In [13]:
movies = movies.drop([19730, 29503, 35587])

In [14]:
movies['id'] = movies['id'].astype('int')

In [15]:
s_movies = movies[movies['id'].isin(links_small)].copy(deep=True)

In [16]:
s_movies.shape
s_movies.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"['Animation', 'Comedy', 'Family']",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"['Adventure', 'Fantasy', 'Family']",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"['Romance', 'Comedy']",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"['Comedy', 'Drama', 'Romance']",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.85949,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,['Comedy'],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.38752,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [17]:
s_movies = s_movies.merge(credits, left_on='id', right_on='id', how='left')

In [18]:
s_movies.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"['Animation', 'Comedy', 'Family']",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,False,NaN,65000000,"['Adventure', 'Fantasy', 'Family']",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"['Romance', 'Comedy']",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de..."
3,False,NaN,16000000,"['Comedy', 'Drama', 'Romance']",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.85949,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de..."
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,['Comedy'],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.38752,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de..."


In [19]:
s_movies = s_movies[['id', 'original_title', 'genres', 'cast', 'vote_average']]

In [20]:
s_movies.index

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            9123, 9124, 9125, 9126, 9127, 9128, 9129, 9130, 9131, 9132],
           dtype='int64', length=9133)

In [21]:
s_movies.columns

Index(['id', 'original_title', 'genres', 'cast', 'vote_average'], dtype='object')

In [22]:
s_movies['genres'] = s_movies['genres'].str.strip('[]').str.replace(' ','').str.replace("'",'')

In [23]:
s_movies['genres'] = s_movies['genres'].str.split(',')

In [24]:
s_movies['genres'].head()

0     [Animation, Comedy, Family]
1    [Adventure, Fantasy, Family]
2               [Romance, Comedy]
3        [Comedy, Drama, Romance]
4                        [Comedy]
Name: genres, dtype: object

In [25]:
genres_list = []
for index, row in s_movies.iterrows():
  genres = row["genres"]
  for genre in genres:
    if genre not in genres_list:
      genres_list.append(genre)

In [26]:
genres_list

['Animation',
 'Comedy',
 'Family',
 'Adventure',
 'Fantasy',
 'Romance',
 'Drama',
 'Action',
 'Crime',
 'Thriller',
 'Horror',
 'History',
 'ScienceFiction',
 'Mystery',
 'War',
 'Foreign',
 'Music',
 'Documentary',
 '',
 'Western',
 'TVMovie']

In [27]:
def binary(g_list, pass_list):
  result_list = []
  for genre in pass_list:
    if genre in g_list:
      result_list.append(1)
    else:
      result_list.append(0)
  return result_list

In [28]:
s_movies['genres_bin'] = s_movies['genres'].apply(lambda x: binary(x, genres_list))

In [29]:
s_movies['genres_bin'].head()

0    [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1    [0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2    [0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3    [0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...
4    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: genres_bin, dtype: object

In [30]:
s_movies['cast'].head()

0    [{'cast_id': 14, 'character': 'Woody (voice)',...
1    [{'cast_id': 1, 'character': 'Alan Parrish', '...
2    [{'cast_id': 2, 'character': 'Max Goldman', 'c...
3    [{'cast_id': 1, 'character': "Savannah 'Vannah...
4    [{'cast_id': 1, 'character': 'George Banks', '...
Name: cast, dtype: object

In [31]:
s_movies['cast'] = s_movies['cast'].fillna('[]').apply(literal_eval)

In [32]:
for index,i in zip(s_movies.index,s_movies['cast']):
  temp_list1 = []
  for j in range(len(i)):
    temp_list1.append((i[j]['character'])) 
  s_movies.loc[index,'cast'] = str(sorted(temp_list1[:5]))

In [33]:
s_movies['cast'] = s_movies['cast'].str.strip('[]').str.replace(' ','').str.replace("'",'')

In [34]:
s_movies['cast'] = s_movies['cast'].str.split(',')

In [35]:
s_movies['cast'].head()

0    [BuzzLightyear(voice), Mr.PotatoHead(voice), R...
1    [AlanParrish, JudySheperd, PeterShepherd, Samu...
2    [ArielGustafson, JohnGustafson, MariaSophiaCol...
3    ["BernadineBernieHarris", "GloriaGloMatthews",...
4    [AnnieBanks-MacKenzie, BryanMacKenzie, FranckE...
Name: cast, dtype: object

In [36]:
s_movies.head()

,id,original_title,genres,cast,vote_average,genres_bin
0,862,Toy Story,"[Animation, Comedy, Family]","[BuzzLightyear(voice), Mr.PotatoHead(voice), R...",7.7,"[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,8844,Jumanji,"[Adventure, Fantasy, Family]","[AlanParrish, JudySheperd, PeterShepherd, Samu...",6.9,"[0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,15602,Grumpier Old Men,"[Romance, Comedy]","[ArielGustafson, JohnGustafson, MariaSophiaCol...",6.5,"[0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,31357,Waiting to Exhale,"[Comedy, Drama, Romance]","[""BernadineBernieHarris"", ""GloriaGloMatthews"",...",6.1,"[0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,11862,Father of the Bride Part II,[Comedy],"[AnnieBanks-MacKenzie, BryanMacKenzie, FranckE...",5.7,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [37]:
cast_list = []
for index, row in s_movies.iterrows():
  cast = row["cast"]
  for i in cast:
    if i not in cast_list:
      cast_list.append(i)

In [38]:
cast_list

['BuzzLightyear(voice)',
 'Mr.PotatoHead(voice)',
 'Rex(voice)',
 'SlinkyDog(voice)',
 'Woody(voice)',
 'AlanParrish',
 'JudySheperd',
 'PeterShepherd',
 'SamuelAlanParrish/VanPelt',
 'SarahWhittle',
 'ArielGustafson',
 'JohnGustafson',
 'MariaSophiaColettaRagetti',
 'MaxGoldman',
 'MelanieGustafson',
 '"BernadineBernieHarris"',
 '"GloriaGloMatthews"',
 'MarvinKing',
 'RobinStokes',
 '"SavannahVannahJackson"',
 'AnnieBanks-MacKenzie',
 'BryanMacKenzie',
 'FranckEggelhoffer',
 'GeorgeBanks',
 'NinaBanks',
 'ChrisShiherlis',
 'Lt.VincentHanna',
 'MichaelCheritto',
 'Nate',
 'NeilMcCauley',
 'DavidLarrabee',
 'LinusLarrabee',
 'MaudeLarrabee',
 'Mrs.IngridTyson',
 'SabrinaFairchild',
 'AuntPolly',
 'BeckyThatcher',
 'HuckFinn',
 'MuffPotter',
 'TomSawyer',
 'DarrenFrancisThomasMcCord',
 'JoshuaFoss',
 'MatthewHallmark',
 'Tyler',
 'VizepräsidentDanielBender',
 'AlecTrevelyan',
 'JackWade',
 'JamesBond',
 'NatalyaFyodorovnaSimonova',
 'XeniaOnatopp',
 'A.J.MacInerney',
 'AndrewShepherd',
 

In [39]:
s_movies['cast_bin'] = s_movies['cast'].apply(lambda x: binary(x, cast_list))

In [40]:
s_movies['cast_bin'].head()

0    [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1    [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, ...
2    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, ...
3    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
4    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: cast_bin, dtype: object

In [41]:
s_movies.head()

,id,original_title,genres,cast,vote_average,genres_bin,cast_bin
0,862,Toy Story,"[Animation, Comedy, Family]","[BuzzLightyear(voice), Mr.PotatoHead(voice), R...",7.7,"[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,8844,Jumanji,"[Adventure, Fantasy, Family]","[AlanParrish, JudySheperd, PeterShepherd, Samu...",6.9,"[0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, ..."
2,15602,Grumpier Old Men,"[Romance, Comedy]","[ArielGustafson, JohnGustafson, MariaSophiaCol...",6.5,"[0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, ..."
3,31357,Waiting to Exhale,"[Comedy, Drama, Romance]","[""BernadineBernieHarris"", ""GloriaGloMatthews"",...",6.1,"[0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,11862,Father of the Bride Part II,[Comedy],"[AnnieBanks-MacKenzie, BryanMacKenzie, FranckE...",5.7,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [42]:
def Similarity(movieId1, movieId2):
  a = s_movies.iloc[movieId1]
  b = s_movies.iloc[movieId2]
  genresA = a['genres_bin']
  genresB = b['genres_bin']
  genreDistance = spatial.distance.cosine(genresA, genresB)
  scoreA = a['cast_bin']
  scoreB = b['cast_bin']
  scoreDistance = spatial.distance.cosine(scoreA, scoreB)

  return genreDistance + scoreDistance

In [43]:
Similarity(4,12)

2.0

In [44]:
def getNeighbors(my_movie, K):
  distances = []
  for index, movie in s_movies.iterrows():
    mid2 = s_movies.index[s_movies['id'] == my_movie['id'].values[0]].tolist()[0]
    if index != mid2:
      dist = Similarity(index, mid2)
      distances.append((index, dist))
  distances.sort(key=operator.itemgetter(1))
  neighbors = []
  for x in range(K):
    neighbors.append(distances[x])
  return neighbors

In [45]:
def recommend(num_of_recoms):
  name = input('Enter a movie title: ').lower()
  new_movie = s_movies[s_movies['original_title'].str.lower().str.contains(name)].iloc[0].to_frame().T
  print('Selected Movie: ',new_movie.original_title.values[0])

  K = num_of_recoms
  neighbors = getNeighbors(new_movie, K)
  print('\n******** Recommended Movies ********\n')
  # result = {'Movies': list(), 'Genres': list()}
  for neighbor in neighbors:
    # result['Movies'].append(s_movies.iloc[neighbor[0]]['original_title'])
    # result['Genres'].append(s_movies.iloc[neighbor[0]]['genres'])
    print( s_movies.iloc[neighbor[0]]['original_title']+"\n\tGenres: "+str(s_movies.iloc[neighbor[0]]['genres']).strip('[]').replace(' ','')+"\n")
  # print(pd.DataFrame(result, columns=['Movies', 'Genres']))

In [46]:
st = time.time()
recommend(25) #pass the number of recommendation you want    ####PLEASE TYPE VALID MOVIE NAME    ####it will give error if entered movie is not present in dataset
print("\n\n\n TIME TAKEN: {}".format(time.time() - st))

Enter a movie title: pulp fiction
Selected Movie:  Pulp Fiction

******** Recommended Movies ********

Shallow Grave
	Genres: 'Crime','Thriller'

The Underneath
	Genres: 'Thriller','Crime'

Kalifornia
	Genres: 'Thriller','Crime'

Love and a .45
	Genres: 'Crime','Thriller'

Reservoir Dogs
	Genres: 'Crime','Thriller'

T-Men
	Genres: 'Thriller','Crime'

Cape Fear
	Genres: 'Crime','Thriller'

King of New York
	Genres: 'Thriller','Crime'

The Hitch-Hiker
	Genres: 'Crime','Thriller'

Pacific Heights
	Genres: 'Crime','Thriller'

Get Carter
	Genres: 'Crime','Thriller'

Snatch
	Genres: 'Thriller','Crime'

Unlawful Entry
	Genres: 'Crime','Thriller'

Ocean's Eleven
	Genres: 'Thriller','Crime'

The Formula
	Genres: 'Thriller','Crime'

Swimming Pool
	Genres: 'Thriller','Crime'

The Night of the Hunter
	Genres: 'Thriller','Crime'

Suspect Zero
	Genres: 'Crime','Thriller'

Ocean's Twelve
	Genres: 'Thriller','Crime'

The Interpreter
	Genres: 'Crime','Thriller'

Ocean's Thirteen
	Genres: 'Crime','Thril